In [2]:
import hublib
from hublib.ui import RunAllButton
RunAllButton(label="Initialize", icon='fa-play-circle-o', style='primary', tooltip='Initialize Tool', hide=False)

Button(button_style='primary', description='Initialize', icon='play-circle-o', style=ButtonStyle(), tooltip='Initialize Tool')

In [7]:
from hublib.ui import RunCommand
from hublib.ui import RunAllButton
import hublib.use
%use octave-4.2.0
import oct2py
%load_ext oct2py.ipython
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from hublib.ui import FileUpload
import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np
from decimal import Decimal
%octave pkg load optim

# Create a callback function that will be called when files are uploaded
def mycb(name, val):
    print("CB %s (length = %s bytes)" % (name, len(val)))
    
# this will allow you to select a single file to be uploaded
f = FileUpload("Please upload Raman spectra data file (CSV)", 
               "Raman data files should be uploaded as 2 column CSV files", 
                cb=mycb)
f

The oct2py.ipython extension is already loaded. To reload it, use:
  %reload_ext oct2py.ipython


Box(children=(HTML(value='<p data-toggle="popover" title="Raman data files should be uploaded as 2 column CSV files">Please upload Raman spectra data file (CSV)</p>', layout=Layout(flex='2 1 auto')), FileWidget()), layout=Layout(border='solid 1px lightgray', display='flex', flex_flow='row', justify_content='space-between', width='auto'))

CB 2018022201_8.csv (length = 15296 bytes)


In [19]:
fit_but = widgets.Button(description='Do Fitting')
    
def fit_but_cb(change):
    
    f.save('tmpfile')
    data = pd.read_csv('tmpfile')
    W=data.Column1
    W=np.array(W)
    I_raw=data.Column2
    I_raw=np.array(I_raw)
    I=((I_raw-np.min(I_raw))/np.max(I_raw-np.min(I_raw)));
    
    pG=[np.max(I), 5, 1581.6] #a w b
    pGp=[np.max(I), 18, 2675]
    PG=%octave -i pG,W,I lsqcurvefit(@Single_Lorentz,pG,W,I);
    PGp=%octave -i pGp,W,I lsqcurvefit(@Single_Lorentz,pGp,W,I,[0,0,2000],[max(I),100,3000]);
    PG[1]=np.absolute(PG[1]);PGp[1]=np.absolute(PGp[1]); #FWHM sometimes returns - bc always squared
    IGfit=%octave -i PG,W Single_Lorentz(PG,W);
    IGpfit=%octave -i PGp,W Single_Lorentz(PGp,W);
    IGfit=IGfit[0];IGpfit=IGpfit[0];
    If=IGfit+IGpfit;
    
    Cdat=np.load('Cfits.npy')

    diffs_lin=[];diffs_Gp=[];
    diffs=[];
    for d in range(6):
        
        LG=Cdat[d][0];
        LGp=Cdat[d][1];
        LGfit=%octave -i LG,W Single_Lorentz(LG,W); 
        LGpfit=%octave -i LGp,W Single_Lorentz(LGp,W);
        Lf=(LGfit+LGpfit)
 
        dfGp=np.average(np.absolute(100*(PGp-LGp)/LGp))
        dfG=np.average(np.absolute(100*(PG-LG)/LG))
        df=np.average([dfGp,dfG])
        diffs.append(df)
  
    md=np.min(diffs)
    mdi=np.argmin(diffs)

    mG=Cdat[mdi][0];mGp=Cdat[mdi][1];
    mGfit=%octave -i mG,W Single_Lorentz(mG,W);
    mGpfit=%octave -i mGp,W Single_Lorentz(mGp,W);
    mf=mGfit+mGpfit;mf=mf[0]
    
    with diffsplot:
        clear_output()
        plt.plot(diffs,'kv')
        plt.plot(mdi,md,'gv')
        plt.annotate((round(Decimal(md),2)),xy=(mdi,1.2*md))
        plt.xticks(range(6),('1','2','3','4','5','Graphite'))
        plt.xlabel('# Layers')
        plt.ylabel('$\Delta$ [%]')
        plt.show()
    with datplot:
        clear_output()
        plt.plot(W,I,'b',W,If,'r',W,mf,'g')
        plt.xlabel('$\omega$ $[cm^{-1}]$')
        plt.ylabel('$I_{norm}$ [arb]')
        plt.legend(labels=['Raw','Fit','Test'])
        plt.show()

fit_but.on_click(fit_but_cb)
fit_but

Button(description='Do Fitting', style=ButtonStyle())

In [14]:
diffsplot=widgets.Output();diffsplot

Output()

In [15]:
datplot=widgets.Output();datplot

Output()